## Time series data for power plants

In [1]:
import pandas as pd

In [2]:
%%time
path_eia923 = './eia923/'
df_eia923_y2017 = pd.read_excel(path_eia923 + 'EIA923_Schedules_2_3_4_5_M_10_2017_20DEC2017.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 32.3 s, sys: 310 ms, total: 32.6 s
Wall time: 33.2 s


In [3]:
%%time
#path_eia923 = './eia923/'
df_eia923_y2016 = pd.read_excel(path_eia923 + 'f923_2016/EIA923_Schedules_2_3_4_5_M_12_2016_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 38.9 s, sys: 306 ms, total: 39.2 s
Wall time: 39.6 s


In [4]:
%%time
df_eia923_y2015 = pd.read_excel(path_eia923 + 'f923_2015/EIA923_Schedules_2_3_4_5_M_12_2015_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 38.1 s, sys: 287 ms, total: 38.4 s
Wall time: 38.7 s


In [40]:
%%time
df_eia923_y2014 = pd.read_excel(path_eia923 + 'f923_2014/EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 40 s, sys: 165 ms, total: 40.1 s
Wall time: 40.4 s


In [41]:
%%time
df_eia923_y2013 = pd.read_excel(path_eia923 + 'f923_2013/EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 36.6 s, sys: 160 ms, total: 36.8 s
Wall time: 37.2 s


In [42]:
%%time
df_eia923_y2012 = pd.read_excel(path_eia923 + 'f923_2012/EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 33.1 s, sys: 69.9 ms, total: 33.2 s
Wall time: 33.3 s


In [52]:
%%time
df_eia923_y2011 = pd.read_excel(path_eia923 + 'f923_2011/EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [2, 3], skiprows = [0, 1])

CPU times: user 45.6 s, sys: 322 ms, total: 45.9 s
Wall time: 47.6 s


In [9]:
%%time
df_eia923_y2010 = pd.read_excel(path_eia923 + 'f923_2010/SCHEDULE 3A 5A 8A 8B 8C 8D 8E 8F 2010 on NOV 30 2011.xls', 
                   sheetname='Generator',
                   header = [4], skiprows = [0, 1, 2])

CPU times: user 3.41 s, sys: 35.4 ms, total: 3.45 s
Wall time: 3.47 s


In [10]:
%%time
df_eia923_y2009 = pd.read_excel(path_eia923 + 'f923_2009/SCHEDULE 3A 5A 8A 8B 8C 8D 8E 8F REVISED 2009 04112011.xls', 
                   sheetname='Generator',
                   header = [4], skiprows = [0, 1, 2])

CPU times: user 2.18 s, sys: 16.9 ms, total: 2.19 s
Wall time: 2.23 s


In [11]:
%%time
df_eia923_y2008 = pd.read_excel(path_eia923 + 'f923_2008/SCHEDULE 3A 5A 8A 8B 8C 8D 8E 8F 2008.xlsm', 
                   sheetname='Generator Data',
                   header = [4], skiprows = [0, 1, 2])

CPU times: user 7.64 s, sys: 26.3 ms, total: 7.67 s
Wall time: 7.71 s


In [12]:
%%time
df_eia923_y2007 = pd.read_excel(path_eia923 + 'f906920_2007/f906920_2007.xls', 
                   sheetname='Page 1 Generation and Fuel Data',
                   header = [4], skiprows = [0, 1])

CPU times: user 4.15 s, sys: 34 ms, total: 4.18 s
Wall time: 4.23 s


In [36]:
def clean_by_year(raw_df, year_str, delim = '\n', state_col = 'Plant State'):
    # Set columns to be single level
    raw_df.columns = raw_df.columns.get_level_values(1)
    # Drop the rows that don't correspond to a specific operator
    drop_row_ind = raw_df.loc[:, 'Operator Id'] == 99999
    df = raw_df.loc[~drop_row_ind, :]
    # Use plant's & operator's id and name as row index
    df = df.set_index(['Plant Id', 'Plant Name', 'Operator Id', 'Operator Name', state_col])
    # Select columns that contains generation
    col_ind = [i for i in df.columns if 'Netgen' in i]
    df = df.loc[:, col_ind]
    # Rename columns to append year
    col_new_name = [i.split(delim)[1][:3].upper() + '_' + year_str for i in df.columns]
    df.columns = col_new_name
    return df

In [14]:
df2017_new = clean_by_year(df_eia923_y2017, '2017')

In [15]:
df2017_new.head(7)

JAN_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            260675   
                                                 AL            480271   
                                                 AL           94891.6   
                                                 AL           5795.41   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             13663   
                                                 AL                 0   

                                                             FEB_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            243163   
                                                 AL            452064   
                                                 AL            194469   
                                                 AL           2461.45   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             14303   
                                                 AL                 0   

                                                             MAR_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            219102   
                                                 AL            405400   
                                                 AL            138007   
                                                 AL           8044.66   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             21736   
                                                 AL                 0   

                                                             APR_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            258366   
                                                 AL            468974   
                                                 AL            111695   
                                                 AL           3016.82   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             17929   
                                                 AL                 0   

                                                             MAY_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            260648   
                                                 AL            481976   
                                                 AL            126550   
                                                 AL           26052.8   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             15656   
                                                 AL                 0   

                                                             JUN_2017  \
Plant Id Plant Name Operator Id Operator Name    Plant State            
3        Barry      195         Alabama Power Co AL            251955   
                                                 AL            460433   
                                                 AL            365095   
                                                 AL            3951.9   
7        Gadsden    195         Alabama Power Co AL                 0   
                                                 AL             15054   
                                                 AL                 0   

                                                             JUL_2017  \
Plant Id Plant Name Operator Id Operator

In [16]:
df2016_new = clean_by_year(df_eia923_y2016, '2016')

In [17]:
df2016_new.head(7)

JAN_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -77.112   
3        Barry              195         Alabama Power Co AL            268797   
                                                         AL            511773   
                                                         AL            329513   
                                                         AL           13195.6   
4        Walter Bouldin Dam 195         Alabama Power Co AL            139641   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     FEB_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -69.679   
3        Barry              195         Alabama Power Co AL            251423   
                                                         AL            457142   
                                                         AL            259648   
                                                         AL           8830.87   
4        Walter Bouldin Dam 195         Alabama Power Co AL            126354   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     MAR_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -48.374   
3        Barry              195         Alabama Power Co AL            214816   
                                                         AL            396211   
                                                         AL            291745   
                                                         AL           1791.39   
4        Walter Bouldin Dam 195         Alabama Power Co AL            112090   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     APR_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -24.341   
3        Barry              195         Alabama Power Co AL            261564   
                                                         AL            490439   
                                                         AL           35269.8   
                                                         AL           1435.23   
4        Walter Bouldin Dam 195         Alabama Power Co AL             51261   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     MAY_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -11.476   
3        Barry              195         Alabama Power Co AL            223662   
                                                         AL            421779   
                                                         AL            376727   
                                                         AL           6305.06   
4        Walter Bouldin Dam 195         Alabama Power Co AL             10233   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     JUN_2016  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           -11.441   
3        Barry              195         Alabama Power Co AL            206449   
                                                         AL            388783   
              

In [18]:
df2015_new = clean_by_year(df_eia923_y2015, '2015')

In [19]:
df2015_new.head(7)

JAN_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           3282.92   
3        Barry              195         Alabama Power Co AL            265432   
                                                         AL            473311   
                                                         AL           61865.9   
                                                         AL           3225.07   
4        Walter Bouldin Dam 195         Alabama Power Co AL             75669   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     FEB_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL            2109.5   
3        Barry              195         Alabama Power Co AL            243437   
                                                         AL            426204   
                                                         AL            182674   
                                                         AL           7474.59   
4        Walter Bouldin Dam 195         Alabama Power Co AL             51335   
7        Gadsden            195         Alabama Power Co AL           11046.8   

                                                                     MAR_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL            3049.8   
3        Barry              195         Alabama Power Co AL            169559   
                                                         AL            289419   
                                                         AL            430269   
                                                         AL           4504.35   
4        Walter Bouldin Dam 195         Alabama Power Co AL             74420   
7        Gadsden            195         Alabama Power Co AL            4597.9   

                                                                     APR_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           3058.66   
3        Barry              195         Alabama Power Co AL             92587   
                                                         AL            161267   
                                                         AL            487290   
                                                         AL           1717.86   
4        Walter Bouldin Dam 195         Alabama Power Co AL             78649   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     MAY_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           1110.17   
3        Barry              195         Alabama Power Co AL            192221   
                                                         AL            334991   
                                                         AL            271555   
                                                         AL           10622.4   
4        Walter Bouldin Dam 195         Alabama Power Co AL             19113   
7        Gadsden            195         Alabama Power Co AL                 0   

                                                                     JUN_2015  \
Plant Id Plant Name         Operator Id Operator Name    Plant State            
2        Bankhead Dam       195         Alabama Power Co AL           934.445   
3        Barry              195         Alabama Power Co AL            249746   
                                                         AL            421928   
              

In [44]:
df2014_new = clean_by_year(df_eia923_y2014, '2014')

In [45]:
df2014_new.head()

JAN_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           6926.23   
3        Barry        195         Alabama Power Co AL            243301   
                                                   AL            438126   
                                                   AL            516889   
                                                   AL           23200.9   

                                                               FEB_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           6350.08   
3        Barry        195         Alabama Power Co AL            125480   
                                                   AL            215543   
                                                   AL            501362   
                                                   AL           12893.5   

                                                               MAR_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           4493.96   
3        Barry        195         Alabama Power Co AL            256678   
                                                   AL            464060   
                                                   AL            144531   
                                                   AL           3984.69   

                                                               APR_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           4749.09   
3        Barry        195         Alabama Power Co AL            241528   
                                                   AL            417970   
                                                   AL            386242   
                                                   AL           8651.54   

                                                               MAY_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           2695.52   
3        Barry        195         Alabama Power Co AL            231508   
                                                   AL            395680   
                                                   AL            575128   
                                                   AL           2930.03   

                                                               JUN_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           2820.77   
3        Barry        195         Alabama Power Co AL            229267   
                                                   AL            388301   
                                                   AL            584798   
                                                   AL           9449.59   

                                                               JUL_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           2122.18   
3        Barry        195         Alabama Power Co AL            246778   
                                                   AL            414002   
                                                   AL            596810   
                                                   AL           13929.1   

                                                               AUG_2014  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           2235.03   
3        Barry        195         Alabama Power Co AL            256702   
                                                   AL            419994   
      

In [46]:
df2013_new = clean_by_year(df_eia923_y2013, '2013', delim = '_', state_col = 'State')

In [47]:
df2013_new.head()

JAN_2013 FEB_2013  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     23892.1  24121.2   
3        Barry        195         Alabama Power Co AL      192108   236929   
                                                   AL      358682   434304   
                                                   AL      545082   334765   
                                                   AL     30233.9  21610.8   

                                                         MAR_2013 APR_2013  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     20156.5  17547.5   
3        Barry        195         Alabama Power Co AL      263525   197894   
                                                   AL      466253   358093   
                                                   AL      226059   244469   
                                                   AL      5486.4  4203.32   

                                                         MAY_2013 JUN_2013  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     22905.1  14673.9   
3        Barry        195         Alabama Power Co AL      245077   180350   
                                                   AL      440014   325672   
                                                   AL      287541   672545   
                                                   AL     4179.05  8101.91   

                                                         JUL_2013 AUG_2013  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL       20147  13330.4   
3        Barry        195         Alabama Power Co AL      238553   260661   
                                                   AL      416636   437247   
                                                   AL      625283   506486   
                                                   AL     3735.63    20637   

                                                         SEP_2013 OCT_2013  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     10038.3  9972.83   
3        Barry        195         Alabama Power Co AL      238775   238711   
                                                   AL      420814   427139   
                                                   AL      578039   579765   
                                                   AL     4814.12  5506.45   

                                                         NOV_2013 DEC_2013  
Plant Id Plant Name   Operator Id Operator Name    State                    
2        Bankhead Dam 195         Alabama Power Co AL     10268.9  24288.3  
3        Barry        195         Alabama Power Co AL      256685   264211  
                                                   AL      451818   467745  
                                                   AL      143120   219258  
                                                   AL     1674.62  6645.93

In [48]:
df2012_new = clean_by_year(df_eia923_y2012, '2012')

In [49]:
df2012_new.head()

JAN_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           31492.3   
3        Barry        195         Alabama Power Co AL            259641   
                                                   AL            474440   
                                                   AL            198083   
                                                   AL           5866.91   

                                                               FEB_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           21380.9   
3        Barry        195         Alabama Power Co AL            241630   
                                                   AL            441241   
                                                   AL            126692   
                                                   AL           1909.79   

                                                               MAR_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           23745.7   
3        Barry        195         Alabama Power Co AL            180631   
                                                   AL            318716   
                                                   AL            503204   
                                                   AL           12629.5   

                                                               APR_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           4837.91   
3        Barry        195         Alabama Power Co AL            213949   
                                                   AL            380279   
                                                   AL            270922   
                                                   AL            4975.7   

                                                               MAY_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           4284.59   
3        Barry        195         Alabama Power Co AL            262098   
                                                   AL            451059   
                                                   AL            209791   
                                                   AL           1752.06   

                                                               JUN_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           5955.77   
3        Barry        195         Alabama Power Co AL            231680   
                                                   AL            403515   
                                                   AL            662268   
                                                   AL           11849.9   

                                                               JUL_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           7091.15   
3        Barry        195         Alabama Power Co AL            242776   
                                                   AL            425137   
                                                   AL            695765   
                                                   AL           2470.09   

                                                               AUG_2012  \
Plant Id Plant Name   Operator Id Operator Name    Plant State            
2        Bankhead Dam 195         Alabama Power Co AL           10061.2   
3        Barry        195         Alabama Power Co AL            251707   
                                                   AL            441375   
      

In [54]:
df2011_new = clean_by_year(df_eia923_y2011, '2011', '_', state_col = 'State')

In [55]:
df2011_new.head()

JAN_2011 FEB_2011  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     11312.4  9090.91   
3        Barry        195         Alabama Power Co AL      184670   221260   
                                                   AL      343942   404669   
                                                   AL           0        .   
                                                   AL      541287   431468   

                                                         MAR_2011 APR_2011  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL       22447    20390   
3        Barry        195         Alabama Power Co AL      209796   221020   
                                                   AL      372456   385976   
                                                   AL           .        .   
                                                   AL      230265   106697   

                                                         MAY_2011 JUN_2011  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     7243.63  5412.24   
3        Barry        195         Alabama Power Co AL      222745   229455   
                                                   AL      396942   394182   
                                                   AL           .        .   
                                                   AL      437671   676387   

                                                         JUL_2011 AUG_2011  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     5866.36  5621.02   
3        Barry        195         Alabama Power Co AL      238564   255397   
                                                   AL      409266   435796   
                                                   AL           .        .   
                                                   AL      741974   663823   

                                                         SEP_2011 OCT_2011  \
Plant Id Plant Name   Operator Id Operator Name    State                     
2        Bankhead Dam 195         Alabama Power Co AL     7644.49   5745.3   
3        Barry        195         Alabama Power Co AL      244954   254343   
                                                   AL      424276   454200   
                                                   AL           .        .   
                                                   AL      237951   341046   

                                                         NOV_2011 DEC_2011  
Plant Id Plant Name   Operator Id Operator Name    State                    
2        Bankhead Dam 195         Alabama Power Co AL      9753.2  18508.4  
3        Barry        195         Alabama Power Co AL      265446   231626  
                                                   AL      472702   426776  
                                                   AL           .        .  
                                                   AL     43893.9    75863

In [56]:
def clean_by_year_after2011(raw_df, year_str, delim = ' ', search_col = 'Net Generation'):
#     # Set columns to be single level
#     raw_df.columns = raw_df.columns.get_level_values(1)
    # Drop the rows that don't correspond to a specific operator
    drop_row_ind = raw_df.loc[:, 'Operator ID'] == 99999
    df = raw_df.loc[~drop_row_ind, :]
    # Use plant's & operator's id and name as row index
    df = df.set_index(['Plant ID', 'Plant Name', 'Operator ID', 'Operator Name', 'State'])
    # Select columns that contains generation
    col_ind = [i for i in df.columns if search_col in i]
    df = df.loc[:, col_ind]
    # Rename columns to append year
    col_new_name = [i.split(delim)[-1][:3].upper() + '_' + year_str for i in df.columns]
    df.columns = col_new_name
    return df

In [57]:
df2010_new = clean_by_year_after2011(df_eia923_y2010, '2010')

In [58]:
df2010_new.head()

JAN_2010  FEB_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      78917.0   68593.0   
                                                 AL      85706.0   64402.0   
                                                 AL     148723.0  119462.0   
                                                 AL     236939.0  185730.0   
                                                 AL     183871.0  321645.0   

                                                        MAR_2010  APR_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      67106.0   21049.0   
                                                 AL      25651.0    -533.0   
                                                 AL      94151.0   88998.0   
                                                 AL     122060.0  183325.0   
                                                 AL     446449.0  418263.0   

                                                        MAY_2010  JUN_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL       -844.0   47681.0   
                                                 AL      51009.0   61392.0   
                                                 AL      25952.0   82822.0   
                                                 AL     189346.0  186810.0   
                                                 AL     407492.0  395281.0   

                                                        JUL_2010  AUG_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      62073.0   53827.0   
                                                 AL      53552.0   56864.0   
                                                 AL      87452.0  102186.0   
                                                 AL     193521.0  175817.0   
                                                 AL     358953.0  387558.0   

                                                        SEP_2010  OCT_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      19536.0    -986.0   
                                                 AL       5903.0   52280.0   
                                                 AL     108346.0     130.0   
                                                 AL      61257.0   72816.0   
                                                 AL     382436.0  126639.0   

                                                        NOV_2010  DEC_2010  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL       -892.0   19274.0   
                                                 AL      42924.0   39345.0   
                                                 AL       -424.0   48524.0   
                                                 AL     145685.0  120080.0   
                                                 AL     310887.0  341321.0   

                                                         DAT_2010  
Plant ID Plant Name Operator ID Operator Name    State             
3        Barry      195         Alabama Power Co AL      435334.0  
                                                 AL      538495.0  
                                                 AL      906322.0  
                                                 AL     1873386.0  
                                                 AL     4080795.0

In [59]:
df2009_new = clean_by_year_after2011(df_eia923_y2009, '2009')

In [60]:
df2009_new.head()

JAN_2009  FEB_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      39699.0    5594.0   
                                                 AL      40189.0   36613.0   
                                                 AL     121179.0   66128.0   
                                                 AL     188495.0  153052.0   
                                                 AL     489319.0  428547.0   

                                                        MAR_2009  APR_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      13015.0   15858.0   
                                                 AL      33709.0    4860.0   
                                                 AL     108994.0  113553.0   
                                                 AL      35101.0   33924.0   
                                                 AL     452941.0  285453.0   

                                                        MAY_2009  JUN_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      68232.0   12428.0   
                                                 AL       -492.0   21981.0   
                                                 AL     101776.0   81563.0   
                                                 AL     191557.0  116239.0   
                                                 AL     433521.0  389233.0   

                                                        JUL_2009  AUG_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL          0.0    -907.0   
                                                 AL      21565.0   75034.0   
                                                 AL     121822.0  142174.0   
                                                 AL     190398.0  222770.0   
                                                 AL     272470.0  406143.0   

                                                        SEP_2009  OCT_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL       -856.0    6058.0   
                                                 AL      71245.0   39556.0   
                                                 AL     135616.0   60471.0   
                                                 AL     206655.0  115768.0   
                                                 AL     437241.0  438032.0   

                                                        NOV_2009  DEC_2009  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL       -668.0   63455.0   
                                                 AL       -323.0   50094.0   
                                                 AL      97744.0  135373.0   
                                                 AL       -896.0  173484.0   
                                                 AL     429647.0   50554.0   

                                                         DAT_2009  
Plant ID Plant Name Operator ID Operator Name    State             
3        Barry      195         Alabama Power Co AL      221908.0  
                                                 AL      394031.0  
                                                 AL     1286393.0  
                                                 AL     1626547.0  
                                                 AL     4513101.0

In [61]:
df2008_new = clean_by_year_after2011(df_eia923_y2008, '2008')

In [62]:
df2008_new.head()

JAN_2008  FEB_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      96021.0   79256.0   
                                                 AL      92534.0   64027.0   
                                                 AL     161559.0  144746.0   
                                                 AL     257675.0  104074.0   
                                                 AL     182397.0       0.0   

                                                        MAR_2008  APR_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      91687.0   73693.0   
                                                 AL      88346.0   75799.0   
                                                 AL     144587.0    -477.0   
                                                 AL     153412.0  223960.0   
                                                 AL      -3316.0  191412.0   

                                                        MAY_2008  JUN_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      68161.0   85115.0   
                                                 AL      65960.0   80193.0   
                                                 AL      76997.0  137305.0   
                                                 AL     255213.0  205602.0   
                                                 AL     484488.0  401640.0   

                                                        JUL_2008  AUG_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      83212.0   73552.0   
                                                 AL      78610.0   70323.0   
                                                 AL     160621.0  137580.0   
                                                 AL     259910.0  248599.0   
                                                 AL     492063.0  471677.0   

                                                        SEP_2008  OCT_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      78868.0   13220.0   
                                                 AL      75097.0    1399.0   
                                                 AL     136624.0   93794.0   
                                                 AL     246877.0  150877.0   
                                                 AL     448367.0  404187.0   

                                                        NOV_2008  DEC_2008  \
Plant ID Plant Name Operator ID Operator Name    State                       
3        Barry      195         Alabama Power Co AL      48935.0   82277.0   
                                                 AL      42067.0   79150.0   
                                                 AL     117046.0  148507.0   
                                                 AL     145354.0  194071.0   
                                                 AL     455671.0  439523.0   

                                                         DAT_2008  
Plant ID Plant Name Operator ID Operator Name    State             
3        Barry      195         Alabama Power Co AL      873997.0  
                                                 AL      813505.0  
                                                 AL     1458889.0  
                                                 AL     2445624.0  
                                                 AL     3968109.0

In [63]:
df2007_new = clean_by_year_after2011(df_eia923_y2007, '2007', delim = '_', search_col = 'NETGEN')

In [64]:
df2007_new.head()

JAN_2007  \
Plant ID Plant Name   Operator ID Operator Name    State               
2        Bankhead Dam 195         Alabama Power Co AL      11372.719   
3        Barry        195         Alabama Power Co AL          0.000   
                                                   AL     310760.000   
                                                   AL     788376.716   
                                                   AL          0.000   

                                                            FEB_2007  \
Plant ID Plant Name   Operator ID Operator Name    State               
2        Bankhead Dam 195         Alabama Power Co AL       5244.315   
3        Barry        195         Alabama Power Co AL          0.000   
                                                   AL     495051.000   
                                                   AL     569104.991   
                                                   AL          0.000   

                                                            MAR_2007  \
Plant ID Plant Name   Operator ID Operator Name    State               
2        Bankhead Dam 195         Alabama Power Co AL       4233.694   
3        Barry        195         Alabama Power Co AL          0.000   
                                                   AL     302190.000   
                                                   AL     628310.020   
                                                   AL          0.000   

                                                            APR_2007  \
Plant ID Plant Name   Operator ID Operator Name    State               
2        Bankhead Dam 195         Alabama Power Co AL       2404.057   
3        Barry        195         Alabama Power Co AL          0.000   
                                                   AL     441570.000   
                                                   AL     727603.000   
                                                   AL          0.000   

                                                             MAY_2007  \
Plant ID Plant Name   Operator ID Operator Name    State                
2        Bankhead Dam 195         Alabama Power Co AL        2477.036   
3        Barry        195         Alabama Power Co AL           0.000   
                                                   AL      264388.000   
                                                   AL     1168880.338   
                                                   AL           0.000   

                                                             JUN_2007  \
Plant ID Plant Name   Operator ID Operator Name    State                
2        Bankhead Dam 195         Alabama Power Co AL        3201.547   
3        Barry        195         Alabama Power Co AL           0.000   
                                                   AL      355154.000   
                                                   AL     1086180.291   
                                                   AL           0.000   

                                                             JUL_2007  \
Plant ID Plant Name   Operator ID Operator Name    State                
2        Bankhead Dam 195         Alabama Power Co AL        3127.331   
3        Barry        195         Alabama Power Co AL           0.000   
                                                   AL      314378.000   
                                                   AL     1065932.399   
                                                   AL           0.000   

                                                            AUG_2007  \
Plant ID Plant Name   Operator ID Operator Name    State               
2        Bankhead Dam 195         Alabama Power Co AL       4016.523   
3        Barry        195         Alabama Power Co AL          0.000   
                                                   AL     468232.000   
                                                   AL     893741.983   
                                                   AL          0.000   
